In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import PromptTemplate

In [2]:
load_dotenv()
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME") 

In [3]:
llm = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [34]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser 
from typing import List

class EquipmentGroup(BaseModel):
    """
        This class is going to capture one group of equipments on the list 'equipments' under the groupd name 'equipment_group_name'.
    """
    equipment_group_name: str = Field(description="Group name of equipments")
    equipments: List[str] = Field(description="List of all the equipments that belong to group indicated in equipment_group_name. One or more elements from equipment_list fit here")


class EquipmentGroups(BaseModel):
    """
        This class is used to srore the collections/list of EquipmentGroup.
    """
    equipment_groups: List[EquipmentGroup]


output_parser = PydanticOutputParser(pydantic_object = EquipmentGroups)

output_format_instructions = output_parser.get_format_instructions()

equipment_list = 'Boiler, Conveyor, Furnace, Compressor, Mixer, Pump, Generator, Chiller, Valve, Fan, Blower, Heater, Dryer, Cooler, Turbine, Cooler, Turbine, Turbaie, Turvine'

promt_template = """You are an assistant expert in manufactoring area, you need to group the following equipments in groups.

Instructions:
1. Generate groups based on similarity of words of equipment names in equipment_list. Some equipments has been named using synonims
2. Generate a meaningful name for each group
3. Generate the output as follows: 
{output_format_instructions}

equipment_list to process:
{equipment_list}

"""

system_promt = PromptTemplate(template=promt_template, input_variables=['equipment_list', 'output_format_instructions'])

chain = system_promt | llm | output_parser


results = chain.invoke({'equipment_list':equipment_list,
                        'output_format_instructions': output_format_instructions})

In [37]:
results.equipment_groups

[EquipmentGroup(equipment_group_name='Heat Generation', equipments=['Boiler', 'Furnace', 'Heater']),
 EquipmentGroup(equipment_group_name='Material Handling', equipments=['Conveyor', 'Mixer']),
 EquipmentGroup(equipment_group_name='Fluid Movement', equipments=['Compressor', 'Pump', 'Valve', 'Fan', 'Blower']),
 EquipmentGroup(equipment_group_name='Power Generation', equipments=['Generator', 'Turbine']),
 EquipmentGroup(equipment_group_name='Cooling', equipments=['Chiller', 'Cooler']),
 EquipmentGroup(equipment_group_name='Drying', equipments=['Dryer'])]

In [56]:
import pandas as pd
pd.DataFrame([dict(result) for result in results.equipment_groups])

,equipment_group_name,equipments
0,Heat Generation,"[Boiler, Furnace, Heater]"
1,Material Handling,"[Conveyor, Mixer]"
2,Fluid Movement,"[Compressor, Pump, Valve, Fan, Blower]"
3,Power Generation,"[Generator, Turbine]"
4,Cooling,"[Chiller, Cooler]"
5,Drying,[Dryer]
